# GRPH
Given: A collection of DNA strings in FASTA format having total length at most 10 kbp.

Return: The adjacency list corresponding to O3. You may return edges in any order.

In [ ]:
from Bio import SeqIO
import networkx as nx

def GRPH(input_file, k=3):
    sequences = {record.id: str(record.seq) for record in SeqIO.parse(input_file, "fasta")}
    graph = nx.DiGraph() # first we need to create a directed graph to represent the overlap relationships
                         # each node in the graph corresponds to a sequence and edges represent suffix-prefix overlaps

    # now we compare all pairs of sequences to find overlaps
    for seq1_id, seq1_seq in sequences.items():    # for each sequence (source node)
        for seq2_id, seq2_seq in sequences.items(): # it compares with every other sequence (target node)
            if seq1_id != seq2_id and seq1_seq[-k:] == seq2_seq[:k]:  # we check if the suffix of seq1 matches the prefix of seq2 with length k and we make sure that seq1 != seq2 to prevent self-loops
                graph.add_edge(seq1_id, seq2_id) # then we add a directed edge from seq1 to seq2
    
    adjacency = []    # the adjacency list contains all directed edges 
    for edge in graph.edges():
        adjacency.append(f"{edge[0]} {edge[1]}")
    
    return adjacency

input_file = "rosalind_grph.txt" 
k = 3  # we need to specify the overlap length

adjacency = GRPH(input_file, k)
for edge in adjacency:
    print(edge)

# TREE
Given: A positive integer n (n≤1000) and an adjacency list corresponding to a graph on n nodes that contains no cycles.

Return: The minimum number of edges that can be added to the graph to produce a tree.

In [ ]:
import networkx as nx

def TREE(input_file):
    with open(input_file, 'r') as file:
        input = file.readlines()
    
    nodes = int(input[0].strip()) # number of nodes given by the input
    
    graph = nx.Graph() # we initialize an undirected graph
    graph.add_nodes_from(range(1, nodes + 1)) # then we add the nodes to the graph (from 1 to 'nodes')
    
    # we add the edges from the adjacency list (each line is an edge)
    for line in input[1:]: 
        u, v = map(int, line.strip().split())
        graph.add_edge(u, v)
    
    components = nx.number_connected_components(graph)  # we determine the n of connected components (a subgraph in which any two nodes are connected by a path)
    min_edges = components - 1 # then we compute the minimum number of edges to connect all components into a tree
    
    return min_edges

input_file = 'rosalind_tree.txt'
result = TREE(input_file)
print(result)

# LONG
Given: At most 50 DNA strings of approximately equal length, not exceeding 1 kbp, in FASTA format (which represent reads deriving from the same strand of a single linear chromosome).

The dataset is guaranteed to satisfy the following condition: there exists a unique way to reconstruct the entire chromosome from these reads by gluing together pairs of reads that overlap by more than half their length.

Return: A shortest superstring containing all the given strings (thus corresponding to a reconstructed chromosome).

In [ ]:
from Bio import SeqIO

def overlap(seq1, seq2):
    combine_str = [] # merged strings
    overlap_str = [] # overlapping strings

    # we check if the suffix of seq1 matches the prefix of seq2
    for i in range(len(seq1)):
        if seq1[i:] == seq2[:len(seq1)-i]:  # overlap
            overlap_str.append(seq1[i:])    # add the overlapping part
            combine_str.append(seq1 + seq2[len(seq1)-i:]) # merge the strings
            break

    # we check if the suffix of s2 matches the prefix of s1
    for i in range(len(seq2)):
        if seq2[i:] == seq1[:len(seq2)-i]:  # overlap 
            overlap_str.append(seq2[i:])
            combine_str.append(seq2 + seq1[len(seq2)-i:])
            break

    if not overlap_str:
        return "", ""  # no overlap found, we return empty values

    # then we rturn the shortest merged string and the longest overlap string
    min_merge = min(combine_str, key=len)  
    max_overlap = max(overlap_str, key=len) 
    return min_merge, max_overlap

def superstring(strings):
    list = strings

    while len(list) > 1:
        most_overlap = ""
        most_pair = []
        most_length = 0

        # we iterate over all pairs of strings to find the pair with the maximum overlap
        for i in range(len(list)-1):
            for j in range(i+1, len(list)):
                min_merge, max_overlap = overlap(list[i], list[j])
                if len(max_overlap) > most_length:
                    most_overlap = min_merge
                    most_pair = [list[i], list[j]]
                    most_length = len(max_overlap)

        # we remove the overlapping pair 
        list.remove(most_pair[0])
        list.remove(most_pair[1])
        list.append(most_overlap) # and add the merged string

    # the final remaining string is the shortest superstring
    return list[0]

def LONG(input_file):
    seq_str = [str(record.seq) for record in SeqIO.parse(input_file, "fasta")]
    return superstring(seq_str)

input_file = "rosalind_long.txt"
result = LONG(input_file)
print(result)

# DEG
In an undirected graph, the degree d(u) of a vertex u is the number of neighbors u has, or equivalently, the number of edges incident upon it.

Given: A simple graph with n≤10^3 vertices in the edge list format.

Return: An array D[1..n] where D[i] is the degree of vertex i.

In [ ]:
def DEG(input_file):
    with open(input_file, 'r') as file:
        vertices, edges = map(int, file.readline().split()) # first we read the n of vertices and edges from the graph in the file
        
        # then we initialize an array (with zeros) to store the degree of each vertex
        deg = [0] * vertices  

        # we have to process each edge and update the degree for both vertices
        for line in file:
            u, v = map(int, line.split())
            deg[u - 1] += 1 
            deg[v - 1] += 1

    return deg # this returns the degrees as a string so in the print() I added *

input_file = 'rosalind_deg.txt'  
result = DEG(input_file)
print(*result)

# DDEG
Given: A simple graph with n≤10^3 vertices in the edge list format.

Return: An array D[1..n] where D[i] is the sum of the degrees of i's neighbors.

In [ ]:
def DDEG(input_file):
    with open(input_file, 'r') as file:
        vertices, edges = map(int, file.readline().split()) # first we read the n of vertices and edges from the graph in the file
        
        # then we initialize an array to store the degree of each vertex
        deg = [0] * vertices  
        adjacency = [[] for i in range(vertices)]
        
        # then we need to process each edge and update the degree and adjacency list for both vertices
        for line in file:
            u, v = map(int, line.split())
            u -= 1
            v -= 1
            
            # here we update degree for both vertices
            deg[u] += 1
            deg[v] += 1
            
            # we add each vertex to the other's adjacency list
            adjacency[u].append(v)
            adjacency[v].append(u)
        
        # we need to compute the sum of the degrees of the neighbors for each vertex
        result = []
        for i in range(vertices):
            neighbor_degree = sum(deg[neighbor] for neighbor in adjacency[i])
            result.append(neighbor_degree)

    return result

input_file = 'rosalind_ddeg.txt'  
result = DDEG(input_file)
print(*result)

# CC
Given: A simple graph with n≤10^3 vertices in the edge list format.

Return: The number of connected components in the graph.

In [ ]:
def filereader(input_file):
    with open(input_file, 'r') as f:
        vertices, n_edges = map(int, f.readline().split())  # we read the n of vertices and edges
        edges = [tuple(map(int, line.split())) for line in f]  # we read all edges
    return vertices, n_edges, edges

def search(node, graph, visit):
    visit[node] = True              # we mark the the current node as visited
    for neighbor in graph[node]:    # then we explore the neighbors
        if not visit[neighbor]:     # if a neighbor hasn't been visited we recursively perform the depth-first search
            search(neighbor, graph, visit)

def CC(vertices, edges):
    graph = {i: [] for i in range(1, vertices + 1)} # we create the graph as an adjacency list
    for u, v in edges:
        graph[u].append(v)
        graph[v].append(u)  

    visit = [False] * (vertices + 1)  # then we create an array to keep track of visited nodes
    counter = 0  # count of connected components

    # then we run the depth-first search for each unvisited node to identify connected components
    for node in range(1, vertices + 1):
        if not visit[node]: # if the current node hasn't been visited we perform the depth-first search from this node
            search(node, graph, visit)
            counter += 1  # and we increment the counter for a new connected component

    return counter

input_file = 'rosalind_cc.txt'  
vertices, n_edges, edges = filereader(input_file)
result = CC(vertices, edges)
print(result)

# BFS
Given: A simple directed graph with n≤10^3 vertices in the edge list format.

Return: An array D[1..n] where D[i] is the length of a shortest path from the vertex 1 to the vertex i (D[1]=0). 
If i is not reachable from 1 set D[i] to −1.

In [ ]:
import networkx as nx

def BFS(input_file):
    graph = nx.DiGraph()         # we first create an empty directed graph

    with open(input_file, 'r') as file:
        vertices, edges = map(int, file.readline().split())
        
        # now we add the edges to the graph
        for line in file:
            u, v = map(int, line.split())
            graph.add_edge(u, v)

    # we need to initialize the distances (-1 means unreachable)
    dist = [-1] * vertices 
    dist[0] = 0  # because the distance from vertex 1 to itself is 0

    visit = [1]  # we start the breadth-first search from vertex 1 (1-based index, so we add 1)
    
    while visit:
        node = visit.pop(0)  # we pop the first element from 'visit'
        
        # we need to check all neighbors of the current node
        for neighbor in graph.neighbors(node):
            if dist[neighbor - 1] == -1:  # if the neighbor has not been visited
                dist[neighbor - 1] = dist[node - 1] + 1
                visit.append(neighbor)  # we add the neighbor to 'visit'
    
    return dist

input_file = 'rosalind_bfs.txt' 
result = BFS(input_file)
print(*result)

# DIJ
Given: A simple directed graph with positive edge weights from 1 to 10^3 and n≤10^3 vertices in the edge list format.

Return: An array D[1..n] where D[i] is the length of a shortest path from the vertex 1 to the vertex i (D[1]=0). If i is not reachable from 1 set D[i] to −1.

In [ ]:
def DIJ(input_file):
    with open(input_file, 'r') as file:
        lines = file.readlines()
    vertices, n_edges = map(int, lines[0].split()) 
    edges = [tuple(map(int, line.split())) for line in lines[1:]]

    # we initialize the graph as an adjacency list
    graph = {node: [] for node in range(1, vertices + 1)}
    for start, end, weight in edges:
        graph[start].append((end, weight))  # then we add the directed edge from start to end with weight

    dist = {node: float('inf') for node in range(1, vertices + 1)} # we initialize  the distances by putting infinity for all vertices except the source (vertex 1)
    dist[1] = 0  # cause the distance from vertex 1 to itself is 0
    list = [(0, 1)]  # a list to store tuples of (vertix, distance) and we start with vertex 1, distance 0
    
    while list:
        current_dist, current_node = list.pop(0) # we pop the node with the smallest known distance
        
        if current_dist > dist[current_node]:  # we move on if the current distance is not the smallest known distance for the node
            continue

        # then we update the distances for all neighbors of the current node
        for neighbor, weight in graph[current_node]:
            new_dist = current_dist + weight
            if new_dist < dist[neighbor]:
                dist[neighbor] = new_dist
                list.append((new_dist, neighbor))
                list = sorted(list)  
                
    # in the result we replace unreachable distances (inf) with -1
    result = [dist[node] if dist[node] != float('inf') else -1 for node in range(1, vertices + 1)]   
    return result

input_file = 'rosalind_dij.txt'  
result = DIJ(input_file)
print(" ".join(map(str, result)))

# DAG
Given: A positive integer k≤20 and k simple directed graphs in the edge list format with at most 10^3 vertices and 3⋅10^3 edges each.

Return: For each graph, output "1" if the graph is acyclic and "-1" otherwise.

In [ ]:
import networkx as nx

def DAG(input_file):
    with open(input_file, 'r') as file:
        lines = file.readlines()
    n_graphs = int(lines[0].strip()) # we read the n of graphs

    result = []
    idx = 1

    for i in range(n_graphs):                                  # we need to process each graph in the file
        while idx < len(lines) and lines[idx].strip() == "":   # we skip any empty lines to reach the next graph's data
            idx += 1

        vertices, edges = map(int, lines[idx].strip().split()) # we read the n of vertices and edges
        idx += 1

        list_edges = []
        for i in range(edges):
            u, v = map(int, lines[idx].strip().split())
            list_edges.append((u, v)) # we append each edge (parsed) as a tuple to the list 
            idx += 1

        graph = nx.DiGraph()
        graph.add_edges_from(list_edges) # we add our list of edges to the graph

        # and we check if the graph is acyclic or not
        if nx.is_directed_acyclic_graph(graph):
            result.append("1")
        else:
            result.append("-1")

    return " ".join(result)

input_file = "rosalind_dag.txt"
result = DAG(input_file)
print(result)

# BIP
Given: A positive integer k≤20 and k simple graphs in the edge list format with at most 10^3vertices each.

Return: For each graph, output "1" if it is bipartite and "-1" otherwise.

In [ ]:
import networkx as nx

def BIP(input_file):
    with open(input_file, 'r') as file:
        lines = file.readlines()
    n_graphs = int(lines[0].strip())

    result = []
    idx = 1

    for i in range(n_graphs):
        while idx < len(lines) and lines[idx].strip() == "":
            idx += 1

        vertices, edges = map(int, lines[idx].strip().split()) # we read the n of vertices and edges
        idx += 1
        list_edges = []
        for i in range(edges):
            u, v = map(int, lines[idx].strip().split())
            list_edges.append((u, v)) # we add the edges to the list as a tuple
            idx += 1

        graph = nx.Graph()
        graph.add_edges_from(list_edges) # we add our list of edges to a graph

        # and we check if the graph is bipartite or not
        if nx.is_bipartite(graph):
            result.append("1")
        else:
            result.append("-1")

    return " ".join(result)

input_file = "rosalind_bip.txt"
result = BIP(input_file)
print(result)

# BF
Given: A simple directed graph with integer edge weights from −10^3 to 10^3 and n≤10^3 vertices in the edge list format.

Return: An array D[1..n] where D[i] is the length of a shortest path from the vertex 1 to the vertex i (D[1]=0). If i is not reachable
from 1 set D[i] to x.

In [ ]:
def BF(input_file):
    with open(input_file, 'r') as file:
        lines = file.readlines()
    vertices, n_edges = map(int, lines[0].strip().split())

    edges = []  # we initialize an empty list to store the edges (u, v, weight)
    for line in lines[1:]:
        u, v, weight = map(int, line.strip().split())
        edges.append((u, v, weight))

    # we initialize a list to store the shortest distances and set all distances to infinity at first
    dist = [float('inf')] * (vertices + 1)
    dist[1] = 0  # cause the distance to the source vertex is 0

    # this is the Bellman-Ford algorithm
    for i in range(vertices - 1):
        for u, v, weight in edges:
            if dist[u] != float('inf') and dist[u] + weight < dist[v]: # we check if the distance to vertex u is not infinity (it has been visited) and if a shorter path is found
                dist[v] = dist[u] + weight                             # we 'relax' the edge by updating the distance to vertex v

    # now we need to format the output and replace the unreachable vertices with 'x'
    result = []
    for i in range(1, vertices + 1):
        if dist[i] == float('inf'): 
            result.append('x')
        else:
            result.append(str(dist[i]))

    return " ".join(result)

input_file = "rosalind_bf.txt"
result = BF(input_file)
print(result)

# CTE
Given: A positive integer k≤20 and k simple directed graphs with positive integer edge weights and at most 10^3 vertices in the edge
list format.

Return: For each graph, output the length of a shortest cycle going through the first specified edge if there is a cycle and "-1"
otherwise.

In [ ]:
def CTE(input_file):
    with open(input_file, 'r') as file:
        lines = file.readlines()

    n_graphs = int(lines[0].strip())
    result = []
    idx = 1

    for i in range(n_graphs):                                # we loop through each graph
        while idx < len(lines) and lines[idx].strip() == "": # we skip empty lines between graphs
            idx += 1

        vertices, n_edges = map(int, lines[idx].strip().split()) # we read the n of vertices and edges for the current graph
        idx += 1

        edges = []
        graph = [[] for i in range(vertices + 1)]
        for i in range(n_edges):
            u, v, weight = map(int, lines[idx].strip().split())
            edges.append((u, v, weight))
            graph[u].append((v, weight))
            idx += 1

        # we get the first edge to check if it participates in a cycle
        first_edge = edges[0]
        u1, v1, weight1 = first_edge

        # we temporarily remove the first edge from the graph to check if it creates a cycle later
        graph[u1] = [(v, weight) for v, weight in graph[u1] if v != v1 or weight != weight1]

        dist = [float('inf')] * (vertices + 1)
        dist[v1] = 0
        visit = [v1]

        # we need to use a sort of like BFS approach to find the shortest path
        while visit:
            u = visit.pop(0)
            for neighbor, weight in graph[u]:
                if dist[u] + weight < dist[neighbor]:
                    dist[neighbor] = dist[u] + weight
                    visit.append(neighbor)

        # then we check if a cycle exists
        if dist[u1] == float('inf'):
            result.append("-1")
        else:
            result.append(str(dist[u1] + weight1))

    return " ".join(result)

input_file = "rosalind_cte.txt"
result = CTE(input_file)
print(result)

# NWC
Given: A positive integer k≤20 and k simple directed graphs with integer edge weights from −10^3 to 10^3 and n≤10^3 vertices in the edge list format.

Return: For each graph, output "1" if it contains a negative weight cycle and "-1" otherwise.

In [ ]:

import networkx as nx

def neg_cycle(n, edges):
    graph = nx.DiGraph()
    graph.add_node(-1)                   # first we add a super source node (-1)
    for i in range(1, n + 1):
        graph.add_node(i)
        graph.add_edge(-1, i, weight=0)  # then we connect the super source to all nodes with 0 weight
    graph.add_weighted_edges_from(edges) # lastly we add all edges

    try:
        nx.single_source_bellman_ford_path_length(graph, -1)  # we run Bellman-Ford from super source node
    except nx.NetworkXUnbounded:
        return "1"  # if a negative cycle exists, it will raise a NetworkXUnbounded exception
    return "-1"  # no negative cycle

def NWC(input_file):
    with open(input_file, 'r') as file:
        lines = file.read().splitlines()

    n_graphs = int(lines[0])  
    result = []
    i = 1                   # we start from the next line after the number of graphs
    while i < len(lines):
        vertices, n_edges = map(int, lines[i].split())
        i += 1
        edges = [tuple(map(int, lines[i + j].split())) for j in range(n_edges)]
        i += n_edges
        result.append(neg_cycle(vertices, edges)) # for the current graph, we call the neg_cycle function to check for negative cycles
    
    print(" ".join(result))  

input_file = 'rosalind_nwc.txt'
NWC(input_file)

# SDAG

Given: A weighted DAG with integer edge weights from −103 to 103 and n≤105 vertices in the edge list format.

Return: An array D[1..n] where D[i] is the length of a shortest path from the vertex 1 to the vertex i (D[1]=0). If i is not reachable from 1 set D[i] to x.

In [ ]:
def topological(vertices, adjacency):
    visit = [False] * (vertices + 1)
    order = []

    def depthsearch(v):
        visit[v] = True
        for neighbor, weight in adjacency.get(v, []):  # we use get to handle missing keys
            if not visit[neighbor]:
                depthsearch(neighbor)
        order.append(v)

    for i in range(1, vertices + 1):
        if not visit[i]:
            depthsearch(i)

    return order[::-1]  # we need to reverse it to get topological order

def shortest(vertices, adjacency):
    dist = ['x'] * (vertices + 1) # first we initialize a distance array with 'x' for unreachable vertices
    dist[1] = 0                   # cause the distance to the source vertex is 0

    topo_order = topological(vertices, adjacency) # we get the topological order

    # then we need to  'relax' the edges in the topological order
    for u in topo_order:
        if dist[u] == 'x':  # we want to skip unreachable nodes
            continue
        for v, weight in adjacency.get(u, []):      # we need to use get to handle missing keys
            if dist[v] == 'x' or dist[v] > dist[u] + weight:
                dist[v] = dist[u] + weight

    return dist[1:]  # lastly, we return the distances for vertices 1 to n

def SDAG(input_file):
    with open(input_file, 'r') as file:
        vertices, n_edges = map(int, file.readline().split())
        
        adjacency = {} # we initialize an empty dictionary to create an adjacency list
        for i in range(n_edges):
            u, v, w = map(int, file.readline().split())
            if u not in adjacency:
                adjacency[u] = []        # we need to initialize an empty list if vertex u is not in the dictionary
            adjacency[u].append((v, w))  # and we append the edge to the adjacency list of u
    
    result = shortest(vertices, adjacency)
    print(" ".join(map(str, result)))


SDAG('rosalind_sdag.txt')